(AAEVALEVEC)=

# 2.3 Algoritmos y aplicaciones de eigenvalores y eigenvectores de una matriz

```{admonition} Notas para contenedor de docker:

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

`docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_optimizacion -p 8888:8888 -d palmoreck/jupyterlab_optimizacion:2.1.4`

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

`docker stop jupyterlab_optimizacion`

Documentación de la imagen de docker `palmoreck/jupyterlab_optimizacion:2.1.4` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/optimizacion).

```

---

Nota generada a partir de [liga](https://www.dropbox.com/s/s4ch0ww1687pl76/3.2.2.Factorizaciones_matriciales_SVD_Cholesky_QR.pdf?dl=0).

Los siguientes métodos asumen que la matriz $A \in \mathbb{R}^{n \times n}$ a la que se les desea calcular sus eigenvalores y eigenvectores es **simétrica**.

## Rotaciones de Jacobi para matrices simétricas

Este método produce una secuencia de transformaciones ortogonales de la forma $J_k^TAJ_k$ con el objetivo de hacer "más diagonal" a la matriz $A$.

```{margin}

Los pasos de este algoritmo representan una guía para la implementación. Al describirse los pasos de un algoritmo no implica que se tengan que implementar uno a continuación del otro como se describe. Si una implementación respeta la lógica y al mismo método, entonces pueden seguirse los pasos de una forma distinta.
```

> **Algoritmo de rotaciones de Jacobi para matrices simétricas**
>>
>> **Dada** $A$ simétrica, **definir** $A_0 = A$
>>
>> **Repetir** el siguiente bloque para $k=0,1,2,\dots$
>>> $A_{k+1} = J_k^T A_k J_k$
>>
>> **hasta** convergencia (satisfacer criterio de paro).



La matriz $J_k$ es una transformación de rotación, ver {ref}`transformaciones de rotación <TROT>`, para eliminar un par de entradas (simétricas) en la matriz $A_k$. Esto ayuda a preservar la simetría de la matriz original.

```{admonition} Observación
:class: tip

Obsérvese que $A_{k+1}$ y $A_{k}$ son matrices similares, ver {ref}`similitud <SIMILITUD>`.

```

Para encontrar la forma que debe tener $J_k$ es suficiente considerar el caso $2 \times 2$ y se **asume** que $a_{12} \neq 0$ pues si $a_{12} =0$ entonces no hay que realizar rotación.:

$$
\begin{eqnarray}
J_k^TAJ_k &=&  
\left [
\begin{array}{cc}
c & s\\
-s & c
\end{array}
\right ]
\left [
\begin{array}{cc}
a_{11} & a_{12}\\
a_{12} & a_{22}
\end{array}
\right ]
\left [
\begin{array}{cc}
c & s\\
-s & c
\end{array}
\right ]
\nonumber \\
&=&
\left [
\begin{array}{cc}
c^2a_{11} -2 csa_{12} + s^2a_{22} & c^2a_{12} - cs(a_{22}-a_{11})-s^2a_{12}\nonumber \\
c^2a_{12} - cs(a_{22}-a_{11})-s^2a_{12} & c^2a_{22}+2csa_{12}+s^2a_{11} \nonumber
\end{array}
\right ]
\nonumber 
\end{eqnarray}
$$

donde: $c$ y $s$ representan a $\cos(\theta), \sin(\theta)$ respectivamente y $\theta$ ángulo para rotar.


Si se desea que la entrada $(1,2)$ (equivalentemente por simetría la $(2,1)$) sea cero se debe cumplir:

$$c^2a_{12} - cs(a_{22}-a_{11})-s^2a_{12}=0.$$

Asignando la variable $t = \frac{s}{c}$ (tangente de $\theta$) se obtiene la ecuación cuadrática:

$$1 - t\frac{(a_{22}-a_{11})}{a_{12}} - t^2 = 0.$$

Equivalentemente:

$$t^2 + t\frac{(a_{22}-a_{11})}{a_{12}} - 1 = t^2 + 2\tau t -1 = 0$$


```{margin}

Las funciones $\frac{1}{\tau+\sqrt{\tau^2+1}}$, $\frac{1}{\tau-\sqrt{\tau^2+1}}$ son estrictamente decrecientes para $\tau \geq 0$ y $\tau < 0$ respectivamente lo que corresponde a un único ángulo $\theta \in \left (-\frac{\pi}{4}, \frac{\pi}{4} \right )$.
```

donde: $\tau = \frac{a_{22}-a_{11}}{2a_{12}}$. Las raíces de la ecuación anterior están dadas por:

$$
\begin{eqnarray}
t_1^* &=& -\tau + \sqrt{\tau^2+1}&=&\frac{1}{\tau+\sqrt{\tau^2+1}} \nonumber \\
t_2^* &=& -\tau - \sqrt{\tau^2+1}&=&\frac{1}{\tau-\sqrt{\tau^2+1}} \nonumber
\end{eqnarray}
$$


Se **sugiere** utilizar la raíz de menor magnitud para disminuir errores por redondeo por lo tanto:

$$t^* = \frac{\text{signo}(\tau)}{|\tau| + \sqrt{1+\tau^2}}$$

donde:

$$
\text{signo}(x) = \begin{cases}
1 \text{ si } x>=0\\
-1 \text{ en otro caso}
\end{cases}
$$

Las relaciones entre coseno, seno y tangente permiten obtener sus valores correspondientes:

$$c = \frac{1}{\sqrt{1+t^{*2}}},$$

$$s = ct^*$$

y así tener completamente definida a la matriz $J_k$.

### Ejemplo

Considera: 

$$
A = 
\left [
\begin{array}{cc}
1&2\\
2&1\\
\end{array}
\right ]
$$

y se desea eliminar las entradas $(1,2)$ y $(2,1)$ con una matriz $J$ de rotación. Entonces:

In [1]:
import numpy as np

In [2]:
A = np.array([[1,2],
              [2,1]])

In [3]:
def sign(x):
    if x >=0:
        return 1
    else:
        return -1

In [4]:
tau = (A[1,1]-A[0,0])/(2*A[0,1])
t_star = sign(tau)/(np.abs(tau) + np.sqrt(1+tau**2))

In [5]:
c = 1/np.sqrt(1+t_star**2)
s = c*t_star

In [6]:
J = np.array([[c, s],
              [-s, c]])

In [7]:
J.T@A@J

array([[-1.,  0.],
       [ 0.,  3.]])

## Algoritmo QR

## Método de la potencia

## Aplicaciones y usos

Ver comentario sobre Schur en https://numpy.org/doc/stable/reference/generated/numpy.linalg.eig.html

https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.schur.html#scipy.linalg.schur

In [2]:
from scipy.linalg import schur, eigvals
import numpy as np

In [3]:
A = np.array([[0, 2, 2], [0, 1, 2], [1, 0, 1]])

In [4]:
T, Z = schur(A)

In [5]:
T

array([[ 2.65896708,  1.42440458, -1.92933439],
       [ 0.        , -0.32948354, -0.49063704],
       [ 0.        ,  1.31178921, -0.32948354]])

In [6]:
evalue, evector = np.linalg.eig(T)

In [7]:
with np.printoptions(precision=3, suppress=True):
    print(evector)

[[1.   +0.j    0.497-0.081j 0.497+0.081j]
 [0.   +0.j    0.   +0.451j 0.   -0.451j]
 [0.   +0.j    0.737+0.j    0.737-0.j   ]]


In [8]:
print(evalue)

[ 2.65896708+0.j         -0.32948354+0.80225456j -0.32948354-0.80225456j]


In [9]:
A2 = T[1:3, 1:3]

In [10]:
A2

array([[-0.32948354, -0.49063704],
       [ 1.31178921, -0.32948354]])

In [11]:
evalue, evector = np.linalg.eig(A2)

In [12]:
evalue

array([-0.32948354+0.80225456j, -0.32948354-0.80225456j])